<a href="https://colab.research.google.com/github/Bosy-Ayman/DSAI-402-RL/blob/main/Lab_Quiz_1_B_Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [2]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

# BatmanExit (3)

In [7]:
import numpy as np
import gym
from gym import spaces

class BatmanStealthEnv(gym.Env):
    """
    Custom RL environment: Batman sneaks through a 5×5 Gotham warehouse
    while avoiding the Joker’s guard.

    Grid layout (conceptual):
    -------------------------
    [0, 0, 0, 0, 0]  ← Guard moves randomly
    [0, 0, 0, 0, 0]
    [0, 0, 0, 0, 0]
    [0, 0, 0, 0, 0]
    [B, 0, 0, 0, E]  ← Batman starts at left, exit is at far right (E)

    State: [batman_x, guard_x]
    Actions:
        0 = move left
        1 = stay
        2 = move right

    Rewards:
        +1  → for surviving a step (not caught)
        +10 → if Batman reaches the exit (rightmost cell)
        -10 → if caught by the guard

    Episode ends if Batman is caught or reaches the exit.
    """

    def __init__(self):
        super(BatmanStealthEnv, self).__init__()
        self.grid_size = 5
        self.action_space = spaces.Discrete(3)  # left, stay, right
        self.observation_space = spaces.MultiDiscrete([self.grid_size, self.grid_size])
        self.reset()

    def reset(self, seed=None, options=None):
        """Resets the environment for a new episode."""
        super().reset(seed=seed)
        self.batman_x = 0
        self.guard_x = np.random.randint(0, self.grid_size)
        self.state = np.array([self.batman_x, self.guard_x], dtype=int)
        self.done = False
        return self.state, {}

    # def step(self, action):
    #     """
    #     TODO: Implement the environment dynamics for one time step.

    #     Parameters
    #     ----------
    #     action : int
    #         Batman's chosen action (0=left, 1=stay, 2=right)

    #     Returns
    #     -------
    #     next_state : np.ndarray
    #         The updated state [batman_x, guard_x]
    #     reward : float
    #         Reward for this step
    #     done : bool
    #         True if the episode ends (caught or escaped)
    #     truncated : bool
    #         Always False (not used here)
    #     info : dict
    #         Optional diagnostic info

    #     Notes for Students
    #     ------------------
    #     - Update Batman’s position based on the action.
    #     - Guard moves randomly: choose from {-1, 0, +1}.
    #     - Clamp both positions within [0, grid_size - 1].
    #     - If positions match → caught (reward = -10, done = True).
    #     - If Batman reaches exit (rightmost cell) → reward = +10, done = True.
    #     - Otherwise → reward = +1.
    #     """
    #     # TODO: Implement the logic described above.
    #     pass
    def step(self,action):

      if action == 0:
        self.batman_x = max(0,self.batman_x-1)

      elif action==2:
        self.batman_x =min(self.grid_size-1, self.batman_x + 1)

      if self.batman_x==self.guard_x:
        reward = -10
        self.done = True

      elif self.batman_x == env.grid_size - 1:
        reward = 10
        self.done =  True
      else:
        reward=1
        self.done = False

      obs=np.array([self.batman_x,self.guard_x])
      info={}
      truncated = False
      return  obs,reward,done,truncated, info


    def render(self):
        """Simple text-based rendering of the current grid."""
        grid = ["_"] * self.grid_size
        grid[self.guard_x] = "G"
        grid[self.batman_x] = "B"
        print("".join(grid))

In [8]:
env = BatmanStealthEnv()
obs, _ = env.reset()

done = False
total_reward = 0
step_count = 0

while not done and step_count < 20:
    action = env.action_space.sample()  # random action
    obs, reward, done, truncated, info = env.step(action)
    env.render()
    total_reward += reward
    step_count += 1
    print(f"Step: {step_count}, Action: {action}, Reward: {reward}, Obs: {obs}")

print(f"Episode finished. Total reward: {total_reward}")

B__G_
Step: 1, Action: 1, Reward: 1, Obs: [0 3]
_B_G_
Step: 2, Action: 2, Reward: 1, Obs: [1 3]
__BG_
Step: 3, Action: 2, Reward: 1, Obs: [2 3]
_B_G_
Step: 4, Action: 0, Reward: 1, Obs: [1 3]
__BG_
Step: 5, Action: 2, Reward: 1, Obs: [2 3]
__BG_
Step: 6, Action: 1, Reward: 1, Obs: [2 3]
_B_G_
Step: 7, Action: 0, Reward: 1, Obs: [1 3]
_B_G_
Step: 8, Action: 1, Reward: 1, Obs: [1 3]
__BG_
Step: 9, Action: 2, Reward: 1, Obs: [2 3]
___B_
Step: 10, Action: 2, Reward: -10, Obs: [3 3]
__BG_
Step: 11, Action: 0, Reward: 1, Obs: [2 3]
___B_
Step: 12, Action: 2, Reward: -10, Obs: [3 3]
___B_
Step: 13, Action: 1, Reward: -10, Obs: [3 3]
___B_
Step: 14, Action: 1, Reward: -10, Obs: [3 3]
__BG_
Step: 15, Action: 0, Reward: 1, Obs: [2 3]
__BG_
Step: 16, Action: 1, Reward: 1, Obs: [2 3]
_B_G_
Step: 17, Action: 0, Reward: 1, Obs: [1 3]
B__G_
Step: 18, Action: 0, Reward: 1, Obs: [0 3]
_B_G_
Step: 19, Action: 2, Reward: 1, Obs: [1 3]
_B_G_
Step: 20, Action: 1, Reward: 1, Obs: [1 3]
Episode finished. Tot

# Policy Iteration (2)

In [20]:
import numpy as np
ACTIONS = [0, 1, 2]

def policy_evaluation(env, policy, gamma=0.9, theta=1e-4):
    """
    Evaluate a given policy by iteratively updating the value function.

    Parameters
    ----------
    env : BatmanStealthEnv
        The Batman stealth environment.
    policy : np.ndarray
        Current policy (shape = [grid_size, grid_size]).
    gamma : float
        Discount factor.
    theta : float
        Convergence threshold.

    Returns
    -------
    V : np.ndarray
        Updated value function after policy evaluation.
    """
    V = np.zeros((env.grid_size, env.grid_size))
    while True:
        delta = 0
        for bx in range(env.grid_size):
            for gx in range(env.grid_size):
                v = V[bx, gx]
                action = policy[bx, gx]

                # Simulate one-step transition
                next_bx = np.clip(bx + (action - 1), 0, env.grid_size - 1)
                next_gx = np.clip(gx + np.random.choice([-1, 0, 1]), 0, env.grid_size - 1)

                if next_bx == next_gx:
                    reward, done = -10, True

                    reward, done = 10, True
                else:
                    reward, done = 1, False

                V[bx, gx] = reward if done else reward + gamma * V[next_bx, next_gx]
                delta = max(delta, abs(v - V[bx, gx]))

        if delta < theta:
            break
    return V


def policy_improvement(env, V, policy, gamma=0.9):
    """
    TODO: Implement the policy improvement step for BatmanStealthEnv.

    Parameters
    ----------
    env : BatmanStealthEnv
        The environment.
    V : np.ndarray
        Current value function, shape [grid_size, grid_size].
    policy : np.ndarray
        Current policy, shape [grid_size, grid_size].
    gamma : float, optional
        Discount factor (default=0.9).

    Returns
    -------
    new_policy : np.ndarray
        Improved policy.
    policy_stable : bool
        True if policy did not change (converged).

    Notes for Students
    ------------------
    - For each state [batman_x, guard_x]:
        * Skip terminal states (caught or escaped).
        * For each action (0, 1, 2):
            - Simulate next state and reward.
            - Compute Q(s, a) = reward + gamma * V(next_state)
        * Choose the action with the highest Q(s, a).
    - Compare with old policy to see if it changed.
    """
    ACTIONS = [0, 1, 2]
    action_values=[]
    V = np.zeros((env.grid_size, env.grid_size))
    flag = True
    for bx in range(env.grid_size):
            for gx in range(env.grid_size):
                v = V[bx, gx]
                action = policy[bx, gx]
                action_values = []
                # Simulate one-step transition
                next_bx = np.clip(bx + (action - 1), 0, env.grid_size - 1)
                next_gx = np.clip(gx + np.random.choice([-1, 0, 1]), 0, env.grid_size - 1)
            for a in  ACTIONS:
                if next_bx == next_gx:
                    reward, done = -10, True
                    reward, done = 10, True
                else:
                    reward = 1
                    done = False

                if done:
                  action_values.append(reward)
                else:
                   action_values.append(reward + gamma * V[next_bx, next_gx])

            q = np.argmax(action)
            policy[bx, gx] = q

            if action !=q:
              flag = False
    return policy,flag


def policy_iteration(env, gamma=0.9, theta=1e-4):
    """
    Run full policy iteration until convergence.
    """
    policy = np.random.choice([0, 1, 2], size=(env.grid_size, env.grid_size))
    stable = False

    while not stable:
        V = policy_evaluation(env, policy, gamma, theta)
        new_policy, stable = policy_improvement(env, V, policy, gamma)
        policy = new_policy

    return policy, V

In [21]:
env = BatmanStealthEnv()
final_policy, final_values = policy_iteration(env)

print("Learned Policy:\n", final_policy)
print("Value Function:\n", np.round(final_values, 2))

Learned Policy:
 [[0 2 0 0 0]
 [1 1 0 2 0]
 [0 0 1 2 0]
 [2 0 1 1 0]
 [2 0 0 2 0]]
Value Function:
 [[10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10.]]
